In [ ]:
!pip install datasets
!pip install transformers[sentencepiece]
!pip install git+https://github.com/csebuetnlp/normalizer

In [ ]:
from datasets.dataset_dict import DatasetDict
from datasets import Dataset
from transformers import AutoModelForPreTraining, AutoTokenizer
from normalizer import normalize # pip install git+https://github.com/csebuetnlp/normalizer
import torch
import warnings
warnings.filterwarnings("ignore")
from transformers import DataCollatorForLanguageModeling
from transformers import create_optimizer
import tensorflow as tf
from transformers.keras_callbacks import PushToHubCallback

In [ ]:
from datasets import load_dataset
file=1 # 1---9
dataset = load_dataset(f"shahidul034/text_summarization_dataset{file}",split="train")
print(f"data{file} is loaded!!")

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split:   0%|          | 0/106525 [00:00<?, ? examples/s]

Dataset parquet downloaded and prepared to /root/.cache/huggingface/datasets/shahidul034___parquet/shahidul034--text_summarization_dataset1-b936081dfe440eab/0.0.0/2a3b91fbd88a2c90d1dbbb32b460cf621d31bd5b05b934492fdef7d8d6f236ec. Subsequent calls will reuse this data.
data1 is loaded!!


In [ ]:
dataset = dataset.filter(lambda x: len(x['content'])>=10)

  0%|          | 0/107 [00:00<?, ?ba/s]

In [ ]:
def normalize_fun(ex):
  ex['content']=normalize(ex['content'])
  # ex['title']=normalize(ex['title'])
  return ex

In [ ]:
dataset = dataset.map(normalize_fun,batched=True)

  0%|          | 0/107 [00:00<?, ?ba/s]

AttributeError: ignored

In [ ]:
from datasets.dataset_dict import DatasetDict
from datasets import Dataset
data = Dataset.from_dict({'content':dataset['content']})
print(f"data {file} is ready")
data

data 1 is ready


Dataset({
    features: ['content'],
    num_rows: 106525
})

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("shahidul034/text_generation_tokenizer2")

In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM
from transformers import AutoTokenizer, TFGPT2LMHeadModel, AutoConfig
from transformers import AutoConfig

# config = AutoConfig.from_pretrained("flax-community/gpt2-bengali")
# model = TFGPT2LMHeadModel(config)
# model(model.dummy_inputs)
# model.summary()

In [ ]:
tokenizer.pad_token = tokenizer.eos_token
data_collator = DataCollatorForLanguageModeling(tokenizer, mlm=False, return_tensors="tf")

In [ ]:
def tokenize_function(example):
    return tokenizer(example["content"], truncation=True)

In [ ]:
raw_datasets=data.train_test_split(test_size=0.05)
tokenized_datasets = raw_datasets.map(tokenize_function, batched=True)

  0%|          | 0/102 [00:00<?, ?ba/s]

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


  0%|          | 0/6 [00:00<?, ?ba/s]

In [ ]:
# saved_folder="/content/drive/MyDrive/my research/text generation model"

tf_train_dataset = tokenized_datasets["train"].to_tf_dataset(
      columns=["input_ids", "attention_mask", "labels"],
      collate_fn=data_collator,
      shuffle=True,
      batch_size=8,
  )
tf_eval_dataset = tokenized_datasets["test"].to_tf_dataset(
      columns=["input_ids", "attention_mask", "labels"],
      collate_fn=data_collator,
      shuffle=False,
      batch_size=8,
  )
num_train_steps = len(tf_train_dataset)
optimizer, schedule = create_optimizer(
    init_lr=5e-5,
    num_warmup_steps=1_000,
    num_train_steps=num_train_steps,
    weight_decay_rate=0.01,
  )

model=TFGPT2LMHeadModel.from_pretrained("shahidul034/text_generation_bangla_model")
# model=TFGPT2LMHeadModel.from_pretrained(saved_folder)
model.compile(optimizer=optimizer)
tf.keras.mixed_precision.set_global_policy("mixed_float16")
model.fit(tf_train_dataset, validation_data=tf_eval_dataset)
model.save_pretrained(saved_folder)

You're using a GPT2TokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


All model checkpoint layers were used when initializing TFGPT2LMHeadModel.

All the layers of TFGPT2LMHeadModel were initialized from the model checkpoint at shahidul034/text_generation_bangla_model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFGPT2LMHeadModel for predictions without further training.
No loss specified in compile() - the model's internal loss computation will be used as the loss. Don't panic - this is a common way to train TensorFlow models in Transformers! To disable this behaviour please pass a loss argument, or explicitly pass `loss=None` if you do not want your model to compute a loss.


   35/12650 [..............................] - ETA: 3:52:52 - loss: 2.8266

ResourceExhaustedError: ignored